<a href="https://colab.research.google.com/github/visalikk/CompHbeta/blob/master/Export_ee_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 99% of this material has been adapted from [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<!--NAVIGATION-->
 < [Specialized Algorithms](9_SpecializedAlgorithms.ipynb) | [Contents](index.ipynb) |  

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/9_SpecializedAlgorithms.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<center>
<h1>Google Earth Engine with Python </h1>
<h2> Export Data </h2>
</center>
<h2> Topics:</h2>

1. Exporting Images
2. Exporting Tables and vector data
3. Exporting videos

You can export images, map tiles, tables and video from Earth Engine. The exports can be sent to your Google Drive account, to [Google Cloud Storage](https://cloud.google.com/storage/) or to a new Earth Engine asset.

To use Google Cloud Storage (a fee-based service), you'll need to set up a project, enable billing for the project, and create a storage bucket. See the [Cloud Storage Quickstart](https://cloud.google.com/storage/docs/quickstart-console) page for instructions. See [this guide](https://cloud.google.com/storage/docs/naming) for information on storage bucket naming. Data exported to a Cloud Storage bucket will have the bucket's [default object Access Control List (ACL)](https://cloud.google.com/storage/docs/access-control/create-manage-lists#defaultobjects). You must have write permission for the specified bucket.

The following sections describe each type of export in detail.

## Connecting GEE with Google Services

- **Authenticate to Earth Engine**

In [ ]:
!pip install earthengine-api #earth-engine Python API

In [ ]:
!earthengine authenticate 

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TANUVMNfr0cgHB-6TW4z3t49iUbIo3gn34FlRMnam38&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2AFtc5C6eNhbWYlKTlBmo5efqLYF6fh3BrojggMubhqRl0Ln1AC48g4

Successfully saved authorization token.


- **Authenticate to Google Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


- **Authenticate to Google Cloud**

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Testing the software setup

In [ ]:
# Earth Engine Python API
import ee
ee.Initialize()

In [ ]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'



Folium version: 0.8.3


In [ ]:
print('Folium version: ' + folium.__version__)

NameError: ignored

In [ ]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1. Exporting images

You can export images from Earth Engine in [GeoTIFF](https://trac.osgeo.org/geotiff/) or [TFRecord](https://www.tensorflow.org/api_docs/python/tf/io#TFRecords_Format_Details) format. See [Configuration Parameters](https://developers.google.com/earth-engine/exporting#configuration-parameters) for more output options.


In [ ]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515')\
            .select(['B4', 'B3', 'B2'])

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([103.9577, 1.4126, 103.5540, 1.2548])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()

#Mapdisplay(center,{'landsat':landsat.getMapId()},zoom_start=7)

## to Drive

To export an image to your Drive account, use `Export.image.toDrive()`. For example, to export portions of a Landsat image, define a region to export, then call `ee.batch.Export.image.toDrive`:

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': landsat,
    'description': 'pusgimageToDriveExample',
    'folder':'Example_folder',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()

In [ ]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: MOOS5RALE2PFLTEU4HXFRBVC).
Polling for task (id: MOOS5RALE2PFLTEU4HXFRBVC).
Polling for task (id: MOOS5RALE2PFLTEU4HXFRBVC).


## to Cloud Storage

To export an image to a Google Cloud Storage bucket, use 
`ee.batch.Export.image.toCloudStorage()`. To export the Landsat image in the previous example to Cloud Storage instead of Drive, use:

In [ ]:
outputBucket = 'bag_csaybar' #Change for your Cloud Storage bucket

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toCloudStorage(**{
    'image': landsat,
      'description': 'imageToCOGeoTiffExample',
    'scale': 100,
    'region': geometry.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF',
    'bucket': outputBucket,
    'formatOptions': {'cloudOptimized': True}
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

## to Asset
To export an image to an asset in your Earth Engine assets folder, use `ee.batch.Export.image.toAsset()`. To manage your Earth Engine assets, or check how much of your storage quota is in use, use the [Asset Manager](https://developers.google.com/earth-engine/asset_manager). The following example illustrates exporting portions of a Landsat image using different pyramiding policies for the same band. The pyramiding policy indicates how Earth Engine computes lower-resolution versions of the asset. Learn more about how Earth Engine handles multiple resolutions in the [scale doc](https://developers.google.com/earth-engine/scale).

In [ ]:
# Get band 4 from the Landsat image, copy it.
band4 = landsat.select('B4').rename('b4_mean')\
               .addBands(landsat.select('B4').rename('b4_sample'))\
               .addBands(landsat.select('B4').rename('b4_max'))\

# Export the image to an Earth Engine asset.
task = ee.batch.Export.image.toAsset(**{
  'image': band4,
  'description': 'imageToAssetExample',
  'assetId': 'users/csaybar/exampleExport',
  'scale': 100,
  'region': geometry.getInfo()['coordinates']
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: NPJTL74EAHRDRAECPNW3KGIY).
Polling for task (id: NPJTL74EAHRDRAECPNW3KGIY).
Polling for task (id: NPJTL74EAHRDRAECPNW3KGIY).
Polling for task (id: NPJTL74EAHRDRAECPNW3KGIY).


You can provide a default pyramiding policy for every band that isn't explicitly specified by using the `'.default'` key. You may also pass in just the `'.default'` key. For example, to make all bands default to the 'sample' pyramiding policy, use `{'.default': 'sample'}`.

# 2. Exporting tables and vector data

You can export a `FeatureCollection` as CSV, SHP (shapefile), GeoJSON, KML, KMZ or TFRecord using `Export.table`. The `FeatureCollection` may represent vectors or simply a table of data. In the latter case, the features in the collection will have null geometry.

Note some additional constraints when working with some file formats, including:

- KML: A `FeatureCollection` exported to a KML file will have all the geometries transformed to unprojected (WGS84) coordinates.

- SHP: A `FeatureCollection` exported to a Shapefile must contain features with the same geometry type and projection and must fit within the [Shapefile size limits](https://desktop.arcgis.com/en/arcmap/latest/manage-data/shapefiles/geoprocessing-considerations-for-shapefile-output.htm). Column names are truncated to 10 characters or fewer, and this must not create duplicate column names.

- TFRecord: A `FeatureCollection` exported to a TFRecord file will always be compressed with the GZIP compression type. You always get exactly one TFRecord file. All numbers are coerced to float type.

In [ ]:
# Make a collection of points.
features = ee.FeatureCollection([ee.Feature(
            ee.Geometry.Point([-96.03836059570312, 29.733377498237072]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.02153778076172, 29.70415730620257]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.8395767211914, 29.603013741043565]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.87133407592773, 29.616296493159023]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.66774368286133, 29.572561281277153]),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.64748764038086, 29.5583768530635]),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.6495475769043, 29.48772369705622]),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.64371109008789, 29.445278236181686]),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.7213020324707, 29.387858955310698]),
            {
              "landcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.71752548217773, 29.377837163511735]),
            {
              "landcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.09672546386719, 30.310689872356505]),
            {
              "landcover": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.09938621520996, 30.2996857382589]),
            {
              "landcover": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.08668327331543, 30.309689547574308]),
            {
              "landcover": 1,
              "system:index": "12"
            }),ee.Feature(
            ee.Geometry.Point([-96.08513832092285, 30.374652388663186]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06831550598145, 30.37913227165939]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06973171234131, 30.37905822567535]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06428146362305, 30.380946380744856]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.09028816223145, 30.359471093493593]),
            {
              "landcover": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.08153343200684, 30.365988230863266]),
            {
              "landcover": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.0890007019043, 30.38753612632244]),
            {
              "landcover": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.0670280456543, 30.401010402148156]),
            {
              "landcover": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06925964355469, 30.398937557618673]),
            {
              "landcover": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.83442687988281, 29.733973740106062]),
            {
              "landcover": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.84060668945312, 29.743811219251754]),
            {
              "landcover": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.69572448730469, 29.765867512326544]),
            {
              "landcover": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.72181701660156, 29.774211870825713]),
            {
              "landcover": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.70422172546387, 29.783821922251498]),
            {
              "landcover": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.70868492126465, 29.785162786328367]),
            {
              "landcover": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.66122055053711, 29.78695057715931]),
            {
              "landcover": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.64345359802246, 29.779873718177605]),
            {
              "landcover": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.63701629638672, 29.784492356534923]),
            {
              "landcover": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.5422592163086, 29.787620990488936]),
            {
              "landcover": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.49179077148438, 29.795963536273014]),
            {
              "landcover": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.46672821044922, 29.817114730739224]),
            {
              "landcover": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.45402526855469, 29.79864492114661]),
            {
              "landcover": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.47565460205078, 29.729501839707396]),
            {
              "landcover": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.49556732177734, 29.71757579752616]),
            {
              "landcover": 0,
              "system:index": "23"
            }),ee.Feature(
            ee.Geometry.Point([-96.5313720703125, 30.083354648756124]),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.66046142578125, 30.18787014479982]),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-97.239990234375, 29.75007093080679]),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.91314697265625, 29.609282224143133]),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.68792724609375, 29.82873108891454]),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-97.11090087890625, 29.88351825335318]),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.99554443359375, 30.099989515377835]),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-97.086181640625, 30.17837331070789]),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.53411865234375, 30.441570071519468]),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.86669921875, 30.642638258763267]),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.67718505859375, 30.694611546632277]),
            {
              "landcover": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.48492431640625, 30.52204730013766]),
            {
              "landcover": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.66619873046875, 30.524413269923986]),
            {
              "landcover": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.60302734375, 30.526779182105784]),
            {
              "landcover": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.38604736328125, 30.687525926582914]),
            {
              "landcover": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.95458984375, 30.26618407355883]),
            {
              "landcover": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.7564926147461, 30.257881008009875]),
            {
              "landcover": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.73554992675781, 30.25432233644344]),
            {
              "landcover": 2,
              "system:index": "17"
            })])
center = features.geometry().centroid().getInfo()['coordinates']
center.reverse()
#Mapdisplay(center,{'feature_example':features.getInfo()},zoom_start=2)

In [ ]:
# Make a collection of points.
water = ee.FeatureCollection([ee.Feature(
            ee.Geometry.Point([-96.03836059570312, 29.733377498237072]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.02153778076172, 29.70415730620257]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.8395767211914, 29.603013741043565]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.87133407592773, 29.616296493159023]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.66774368286133, 29.572561281277153]),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.64748764038086, 29.5583768530635]),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.6495475769043, 29.48772369705622]),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.64371109008789, 29.445278236181686]),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.7213020324707, 29.387858955310698]),
            {
              "landcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.71752548217773, 29.377837163511735]),
            {
              "landcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.09672546386719, 30.310689872356505]),
            {
              "landcover": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.09938621520996, 30.2996857382589]),
            {
              "landcover": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.08668327331543, 30.309689547574308]),
            {
              "landcover": 1,
              "system:index": "12"
            })])
center = water.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'feature_example':water.getInfo()},zoom_start=2)

## to Drive
To export a FeatureCollection to your Drive account, use 
ee.batch.Export.table.toDrive(). For example:



In [ ]:
# Export the FeatureCollection to a KML file.
task = ee.batch.Export.table.toDrive(**{
  'collection': features,
  'description':'featuresvectorsToDrive',
  'fileFormat': 'SHP'
})
task.start()

In [ ]:
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
# Make a collection of points.
urban = ee.FeatureCollection([ee.Feature(
            ee.Geometry.Point([-96.08513832092285, 30.374652388663186]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06831550598145, 30.37913227165939]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06973171234131, 30.37905822567535]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06428146362305, 30.380946380744856]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.09028816223145, 30.359471093493593]),
            {
              "landcover": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.08153343200684, 30.365988230863266]),
            {
              "landcover": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.0890007019043, 30.38753612632244]),
            {
              "landcover": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.0670280456543, 30.401010402148156]),
            {
              "landcover": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.06925964355469, 30.398937557618673]),
            {
              "landcover": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.83442687988281, 29.733973740106062]),
            {
              "landcover": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.84060668945312, 29.743811219251754]),
            {
              "landcover": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.69572448730469, 29.765867512326544]),
            {
              "landcover": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.72181701660156, 29.774211870825713]),
            {
              "landcover": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.70422172546387, 29.783821922251498]),
            {
              "landcover": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.70868492126465, 29.785162786328367]),
            {
              "landcover": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.66122055053711, 29.78695057715931]),
            {
              "landcover": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.64345359802246, 29.779873718177605]),
            {
              "landcover": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.63701629638672, 29.784492356534923]),
            {
              "landcover": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.5422592163086, 29.787620990488936]),
            {
              "landcover": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.49179077148438, 29.795963536273014]),
            {
              "landcover": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.46672821044922, 29.817114730739224]),
            {
              "landcover": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.45402526855469, 29.79864492114661]),
            {
              "landcover": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.47565460205078, 29.729501839707396]),
            {
              "landcover": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.49556732177734, 29.71757579752616]),
            {
              "landcover": 0,
              "system:index": "23"
            })])
center = urban.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'feature_example':urban.getInfo()},zoom_start=2)

In [ ]:
# Make a collection of points.
forest = ee.FeatureCollection([ee.Feature(
            ee.Geometry.Point([-96.5313720703125, 30.083354648756124]),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.66046142578125, 30.18787014479982]),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-97.239990234375, 29.75007093080679]),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.91314697265625, 29.609282224143133]),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.68792724609375, 29.82873108891454]),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-97.11090087890625, 29.88351825335318]),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.99554443359375, 30.099989515377835]),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-97.086181640625, 30.17837331070789]),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-96.53411865234375, 30.441570071519468]),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.86669921875, 30.642638258763267]),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.67718505859375, 30.694611546632277]),
            {
              "landcover": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.48492431640625, 30.52204730013766]),
            {
              "landcover": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.66619873046875, 30.524413269923986]),
            {
              "landcover": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.60302734375, 30.526779182105784]),
            {
              "landcover": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.38604736328125, 30.687525926582914]),
            {
              "landcover": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.95458984375, 30.26618407355883]),
            {
              "landcover": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.7564926147461, 30.257881008009875]),
            {
              "landcover": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([-95.73554992675781, 30.25432233644344]),
            {
              "landcover": 2,
              "system:index": "17"
            })])
center = forest.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'feature_example':forest.getInfo()},zoom_start=2)

## to Cloud Storage

To export a `FeatureCollection` to Cloud Storage, use ee.batch.Export.table.toCloudStorage(). For example, using the features defined previously:



In [ ]:
outputBucket = 'bag_csaybar' #Change for your Cloud Storage bucket

# Export a KML file to Cloud Storage.
task = ee.batch.Export.table.toCloudStorage(**{
  'collection': features,
  'description':'vectorsToCloudStorageExample',
  'bucket': outputBucket,
  'fileNamePrefix': 'exampleTableExport',
  'fileFormat': 'KML'    
})

task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: RYELOVWIMRPQC45R3OHDGV3Z).


## to Asset
To export a FeatureCollection as an Earth Engine asset, use `ee.batch.Export.table.toAsset()`. For example, using the features defined previously:

In [ ]:
# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toAsset(**{
    'collection': features,
    'description':'exportToTableAssetExample',
    'assetId': 'users/csaybar/exampleExport'
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: 2Y6LO3NGZAVXFS55QFZODUIO).
Polling for task (id: 2Y6LO3NGZAVXFS55QFZODUIO).
Polling for task (id: 2Y6LO3NGZAVXFS55QFZODUIO).
Polling for task (id: 2Y6LO3NGZAVXFS55QFZODUIO).


There are several limitations on the size and shape of Earth Engine table assets:

- Maximum of 100 million features

- Maximum of 1000 properties (columns)

- Maximum of 100,000 vertices for each row's geometry

- Maximum of 100,000 characters per string value

# 3. Exporting video

To export ordered image collections as video, where frames are defined by images in the collection, use `Export.video()`. You can configure the way the `ImageCollection` is turned into video by setting frame rate, scale and dimensions. The video will be encoded as an MP4.

In [ ]:
# Load a Landsat 5 image collection.
# 1. San Francisco Bay.
# 2. Filter cloudy scenes.
# 3. Get 20 years of imagery.
# 4. Need to have 3-band imagery for the video.
# 5. Need to make the data 8-bit.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))\
               .filter(ee.Filter.lt('CLOUD_COVER', 30))\
               .filterDate('1991-01-01','2011-12-30')\
               .select(['B4', 'B3', 'B2'])\
               .map(lambda img:img.multiply(512).uint8())

# Define an area to export.
polygon = ee.Geometry.Rectangle([-122.7286, 37.6325, -122.0241, 37.9592]);
center = polygon.centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'SF LANDSAT':collection.first().getMapId()})

## to Drive

Export video to your Drive account with `ee.batch.Export.video.toDrive`. For example, the following export makes a video from 20 years of Landsat imagery:

In [ ]:
# Export (change dimensions or scale for higher quality).
task = ee.batch.Export.video.toDrive(**{
    'collection': collection,
    'description': 'sfVideoExample',
    'dimensions': 720,
    'folder':'Example_folder',
    'framesPerSecond': 12,
    'region': polygon.getInfo()['coordinates']
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Note that the frame rate and dimensions can be set from a dictionary of parameters passed to the export. Adjust these parameters to customize the video. Also note that the input ImageCollection is required to have 3-band (RGB), 8-bit images. In this example, the 8-bit, 3-band format is explicitly set. Alternatively, map a function which calls `image.visualize()` over the collection. See [the section on Visualization images](https://developers.google.com/earth-engine/image_visualization#visualization-images) for details. Video exports can take a significant amount of time to complete, so it's not unusual to see the export task running for an extended period.

## to Cloud Storage

To export a video to Cloud Storage, use Export.video.toCloudStorage(). For example, using the ImageCollection from the previous example:

In [ ]:
outputBucket = 'bag_csaybar' #Change for your Cloud Storage bucket

# Export video to cloud storage.
task = ee.batch.Export.video.toCloudStorage(**{
    'collection': collection,
    'description': 'sfVideoExampleToCloud',
    'dimensions': 720,
    'folder':'Example_folder',
    'bucket': outputBucket,
    'framesPerSecond': 12,
    'region': polygon.getInfo()['coordinates']
})
task.start()

In [ ]:
!pip install gee2drive

In [ ]:
import ee
import os
import sys
import gee2drive
[head,tail]=os.path.split(gee2drive.__file__)
os.chdir(head)
sys.path.append(head)
from export import exp


In [ ]:
ee.Initialize()
exp(collection=ee.ImageCollection('LANDSAT/LC08/C01/T1').filterMetadata('CLOUD_COVER','less_than',20),
    folderpath="l8-out",start="2018-02-01",end="2018-06-01",
    geojson=r"/content/drive/boulder.geojson",bandnames="['B4','B3','B2']", operator="bb",typ="ImageCollection")

TypeError: ignored

In [ ]:
// Load a landsat image and select three bands.
var landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515')
  .select(['B4', 'B3', 'B2']);

// Create a geometry representing an export region.
var geometry = ee.Geometry.Rectangle([103.9577, 1.4126, 103.5540, 1.2548]);

// Export the image, specifying scale and region.
Export.image.toDrive({
  image: landsat,
  description: 'PU_sgimageToDriveExample',
  scale: 30,
  region: geometry
});

SyntaxError: ignored

<!--NAVIGATION-->
 < [Specialized Algorithms](9_SpecializedAlgorithms.ipynb) | [Contents](index.ipynb) |  

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/9_SpecializedAlgorithms.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>